### Библиотеки / данные 

импортируем numpy и pandas

In [1]:
import numpy as np
import pandas as pd

настройки pandas

In [2]:
pd.options.display.max_rows = 10

- считываем данные
- используем столбец Symbol в качестве индекса 
- считываем только столбцы ['Symbol', 'Sector', 'Price', 'Book Value']

| Column Name        | Description
| ------------- |:-------------:|
|Symbol|Сокращенное название организации|
|Name|Полное название организации|
|Sector|Сектор экономики|
|Price|Стоимость акции|
|Dividend Yield|Дивидендная доходность|
|Price/Earnings|Цена / прибыль|
|Earnings/Share|Прибыль на акцию|
|Book Value|Балансовая стоимость компании|
|52 week low|52-недельный минимум|
|52 week high|52-недельный максимум|
|Market Cap|Рыночная капитализация|
|EBITDA|**E**arnings **b**efore **i**nterest, **t**axes, **d**epreciation and **a**mortization|
|Price/Sales|Цена / объём продаж|
|Price/Book|Цена / балансовая стоимость|
|SEC Filings|Ссылка *sec.gov*|

In [3]:
sp500 = pd.read_csv("sp500.csv",
                    index_col='Symbol', 
                    usecols=['Symbol', 'Sector', 'Price', 'Book Value'])

### Мотивация

создаем DataFame для примера

In [7]:
np.random.seed(123)
df = pd.DataFrame({'value':np.random.random(10000), 'key':range(100, 10100)})
df.head()

,value,key
0,0.696469,100
1,0.286139,101
2,0.226851,102
3,0.551315,103
4,0.719469,104


отбираем строку, в котором значение столбца key равно 10099

In [8]:
df[df.key==10099]

,value,key
9999,0.613297,10099


измеряем время выполнения операции отбора

In [9]:
%timeit df[df.key==10099]

1000 loops, best of 5: 484 µs per loop


превращаем столбец key в index

In [10]:
df_with_index = df.set_index(['key'])
df_with_index.head()

,value
key,
100,0.696469
101,0.286139
102,0.226851
103,0.551315
104,0.719469


теперь можно найти это значение с помощью индекса

In [11]:
df_with_index.loc[10099]

value    0.613297
Name: 10099, dtype: float64

и теперь операция выполняется намного быстрее

In [12]:
%timeit df_with_index.loc[10099]

The slowest run took 4.45 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 85.9 µs per loop


<center><font color = 'green'><b>Вывод: использование индекса повышает скорость доступа к данным более чем в n раз!</b></font>

### Операции

#### сброс

исследуем несколько строк датафрейма data

In [13]:
sp500.head(3)

,Sector,Price,Book Value
Symbol,,,
MMM,Industrials,141.14,26.668
ABT,Health Care,39.60,15.573
ABBV,Health Care,53.95,2.954


сбрасываем индекс, помещая значения индекса в столбец

In [14]:
index_moved_to_col = sp500.reset_index()
index_moved_to_col.head()

,Symbol,Sector,Price,Book Value
0,MMM,Industrials,141.14,26.668
1,ABT,Health Care,39.60,15.573
2,ABBV,Health Care,53.95,2.954
3,ACN,Information Technology,79.79,8.326
4,ACE,Financials,102.91,86.897


#### установка

а теперь делаем столбец Sector индексом

In [15]:
index_moved_to_col.set_index('Sector').head()

,Symbol,Price,Book Value
Sector,,,
Industrials,MMM,141.14,26.668
Health Care,ABT,39.60,15.573
Health Care,ABBV,53.95,2.954
Information Technology,ACN,79.79,8.326
Financials,ACE,102.91,86.897


#### над множествами

Датафреймы для примера:

In [16]:
data_rnd_part_1 = sp500.sample(100, random_state=333)
data_rnd_part_2 = sp500.sample(100, random_state=777)

In [17]:
data_rnd_part_1.head()

,Sector,Price,Book Value
Symbol,,,
NDAQ,Financials,36.45,36.578
FDO,Consumer Discretionary,56.65,14.122
EBAY,Information Technology,52.02,15.477
ECL,Materials,108.05,24.019
LO,Consumer Staples,60.20,-5.953


In [18]:
data_rnd_part_2.head()

,Sector,Price,Book Value
Symbol,,,
SRCL,Industrials,112.48,20.802
ADP,Information Technology,78.85,13.729
SPLS,Consumer Discretionary,11.65,9.525
COV,Health Care,71.19,21.219
HRL,Consumer Staples,46.66,13.311


объединение:

In [19]:
ind_union = data_rnd_part_1.index | data_rnd_part_2.index
ind_union

Index(['AA', 'ACT', 'ADBE', 'ADP', 'AEE', 'AFL', 'AIZ', 'ALL', 'ALTR', 'ALXN',
       ...
       'VTR', 'VZ', 'WHR', 'WMB', 'WU', 'XOM', 'XRAY', 'XRX', 'ZMH', 'ZTS'],
      dtype='object', name='Symbol', length=177)

пересечение

In [20]:
ind_intersection = data_rnd_part_1.index & data_rnd_part_2.index
len(ind_intersection)

23

разность

In [21]:
ind_diff = data_rnd_part_1.index.difference(data_rnd_part_2.index)
len(ind_diff)

77

### Иерархическая индексация

сначала помещаем символы в столбец

In [22]:
reindexed = sp500.reset_index()

In [23]:
reindexed.head()

,Symbol,Sector,Price,Book Value
0,MMM,Industrials,141.14,26.668
1,ABT,Health Care,39.60,15.573
2,ABBV,Health Care,53.95,2.954
3,ACN,Information Technology,79.79,8.326
4,ACE,Financials,102.91,86.897


а теперь индексируем датафрейм data по столбцам Sector и Symbol

In [24]:
multi_fi = reindexed.set_index(['Sector', 'Symbol'])
multi_fi.head()

Price  Book Value
Sector                 Symbol                    
Industrials            MMM     141.14      26.668
Health Care            ABT      39.60      15.573
                       ABBV     53.95       2.954
Information Technology ACN      79.79       8.326
Financials             ACE     102.91      86.897

наш индекс - это MultiIndex

In [25]:
type(multi_fi.index)

pandas.core.indexes.multi.MultiIndex

он имеет два уровня

In [26]:
len(multi_fi.index.levels)

2

каждый уровень индекса - это индекс

In [27]:
multi_fi.index.levels[1]

Index(['A', 'AA', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACE', 'ACN', 'ACT', 'ADBE',
       ...
       'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YHOO', 'YUM', 'ZION', 'ZMH',
       'ZTS'],
      dtype='object', name='Symbol', length=500)

изменение порядка уровней индекса:

In [28]:
multi_fi.reorder_levels([1, 0], axis=0).head()

,,Price,Book Value
Symbol,Sector,,
MMM,Industrials,141.14,26.668
ABT,Health Care,39.60,15.573
ABBV,Health Care,53.95,2.954
ACN,Information Technology,79.79,8.326
ACE,Financials,102.91,86.897


получаем все акции, которые имеют значение Industrials <br> обратите внимание, что в результатах индекс уровня 0 не выводится 

In [29]:
multi_fi.xs('Industrials').head()

,Price,Book Value
Symbol,,
MMM,141.14,26.668
ALLE,52.46,0.000
APH,95.71,18.315
AVY,48.20,15.616
BA,132.41,19.870


отбираем строки, в которых индекс уровня 1 (Symbol) имеет значение ALLE

In [30]:
multi_fi.xs('ALLE', level=1)

,Price,Book Value
Sector,,
Industrials,52.46,0.0


скомбинируем уровни индексов

In [31]:
multi_fi.xs('Industrials').xs('UPS')

Price         102.73
Book Value      6.79
Name: UPS, dtype: float64

комбинируем уровни индексов, используя кортеж

In [32]:
multi_fi.xs(('Industrials', 'UPS'))

Price         102.73
Book Value      6.79
Name: (Industrials, UPS), dtype: float64

### [Типы](http://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#index-types)

#### Основной тип Index

покажем, что столбцы фактически являются индексом

In [33]:
sp500.columns

Index(['Sector', 'Price', 'Book Value'], dtype='object')

In [34]:
sp500.index

Index(['MMM', 'ABT', 'ABBV', 'ACN', 'ACE', 'ACT', 'ADBE', 'AES', 'AET', 'AFL',
       ...
       'XEL', 'XRX', 'XLNX', 'XL', 'XYL', 'YHOO', 'YUM', 'ZMH', 'ZION', 'ZTS'],
      dtype='object', name='Symbol', length=500)

#### Int64Index и RangeIndex

Явно создаем Int64Index

In [35]:
df_i64 = pd.DataFrame(np.arange(10, 20), index=np.arange(0, 10))
df_i64.head()

,0
0,10
1,11
2,12
3,13
4,14


смотрим индекс

In [36]:
df_i64.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

по умолчанию мы получаем RangeIndex

In [37]:
df_range = pd.DataFrame(np.arange(10, 15))
df_range.head()

,0
0,10
1,11
2,12
3,13
4,14


In [38]:
df_range.index

RangeIndex(start=0, stop=5, step=1)

#### Float64Index

индексы, использующие Float64Index

In [39]:
df_f64 = pd.DataFrame(np.arange(0, 1000, 5), 
                      np.arange(0.0, 100.0, 0.5))
df_f64.head()

,0
0.0,0
0.5,5
1.0,10
1.5,15
2.0,20


In [40]:
df_f64.index

Float64Index([ 0.0,  0.5,  1.0,  1.5,  2.0,  2.5,  3.0,  3.5,  4.0,  4.5,
              ...
              95.0, 95.5, 96.0, 96.5, 97.0, 97.5, 98.0, 98.5, 99.0, 99.5],
             dtype='float64', length=200)